In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_airlines = pd.read_csv('airlines.csv')
df_airports = pd.read_csv('airports.csv')
df_flights = pd.read_csv('flights.csv', low_memory=False)
df_flights_backup = df_flights.copy()

In [3]:
# backup to not rerun the read_csv again
df_flights = df_flights_backup.copy()

In [4]:
df_flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


Let's clean our Datasets first by removing null values.

In [ ]:
df_flights.isnull().sum()

See the distinct values

In [ ]:
for col in df_flights.columns:
    if(len(df_flights[col].unique()) <= 50):
        print(col, df_flights[col].unique())

CANCELLATION_REASON has many null values as it's found only when the flight is cancelled.we can see the values count in it to see why the flight might be cancelled.

A : Airline/Carrier
B : Weather
C : National Air System
D : Security

In [ ]:
sns.countplot(data=df_flights[df_flights['CANCELLATION_REASON'].notnull()], y='CANCELLATION_REASON')

The last 5 columns has many nulls and the same number of nulls let's see that.

In [ ]:
df_flights.loc[df_flights['AIRLINE_DELAY'].notnull()]

It's the delay in time made by AIR_SYSTEM and AIRLINE and WEATHER AND AIRCRAFT they have many null values,
But it's null as the flight might not have delayed so i'll put their null values with 0.0 instead of removing them.

In [ ]:
#yousef+sary :D
df_flights[df_flights.columns[25:31]] = df_flights[df_flights.columns[25:31]].fillna(value=0.0)

We can't drop the rows which contains null in any column as for example the ELAPSED_TIME and AIR_TIME couldn't be recorded as the whole trip is canceled or diverted, let's see if the cancellation affects the elapsed_time and air_time.

In [ ]:
df_flights.loc[(df_flights['CANCELLED'] == True)&(df_flights['ELAPSED_TIME'].notnull())]

So that's true : the values of ELAPSED_TIME and AIR_TIME are missing as the whole trip is cancelled, the same with diverted flights.

In [ ]:
df_flights.loc[(df_flights['DIVERTED'] == True)&(df_flights['ELAPSED_TIME'].notnull())]

We can fill null the rows with 0s which have true in canceled or diverted.

In [ ]:
df_flights[(df_flights['CANCELLED'] == 1) | (df_flights['DIVERTED'] == 1)]=df_flights[(df_flights['CANCELLED'] == 1) | (df_flights['DIVERTED'] == 1)].fillna(0)

Flights contain YEAR which has 1 distinct value 2015. drop it

In [ ]:
df_flights.drop('YEAR', axis=1,inplace=True)

In [ ]:
df_flights.isnull().sum()

In [ ]:
df_airports.info()

In [ ]:
df_airports['COUNTRY'].unique()

Airports has useless column (COUNTRY) which contain 1 distinct value "USA".

In [ ]:
df_airports.drop('COUNTRY', axis=1, inplace=True)

drop the airports that i don't know it's geolocation. 3 rows deleted.

In [ ]:
df_airports.dropna(inplace=True)

Now remove the outliers for airports.

In [ ]:
# yousef khaled
import matplotlib.pyplot as plt 
# Boxplot before removing outliers for LATITUDE
df_airports_sort = df_airports.sort_values(by=['LATITUDE']) #sort the LATITUDE column
df_airports_sort
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
bp = ax.boxplot(df_airports_sort['LATITUDE'])

In [ ]:
#yousef khaled
# Remove outliers for LATITUDE
Q1 = df_airports_sort['LATITUDE'].quantile(0.25)
Q3 = df_airports_sort['LATITUDE'].quantile(0.75)
IQR = Q3 - Q1
filter_mask = (df_airports_sort['LATITUDE'] >= Q1 - 1.5*IQR) & (df_airports_sort['LATITUDE'] <= Q3 + 1.5*IQR)
outliersToBeDropped =df_airports_sort.shape[0] - df_airports_sort.loc[filter_mask].shape[0]
print(f'we will drop {outliersToBeDropped} rows')
df_airports_sort = df_airports_sort.loc[filter_mask]

In [ ]:
#yousef khaled

# Boxplot before removing outliers for LONGITUDE
df_airports_sort = df_airports.sort_values(by=['LONGITUDE']) #sort the LONGITUDE column
df_airports_sort
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
bp = ax.boxplot(df_airports_sort['LONGITUDE'])



In [ ]:
#yousef khaled
# Remove outliers for LONGITUDE
Q1 = df_airports_sort['LONGITUDE'].quantile(0.25)
Q3 = df_airports_sort['LONGITUDE'].quantile(0.75)
IQR = Q3 - Q1
filter_mask = (df_airports_sort['LONGITUDE'] >= Q1 - 1.5*IQR) & (df_airports_sort['LONGITUDE'] <= Q3 + 1.5*IQR)
outliersToBeDropped =df_airports_sort.shape[0] - df_airports_sort.loc[filter_mask].shape[0]
print(f'we will drop {outliersToBeDropped} rows')
df_airports_sort = df_airports_sort.loc[filter_mask]

Now remove the outliers for flights.

In [ ]:
#yousef khaled
# Boxplot before removing outliers for DISTANCE
df_flights = df_flights.sort_values(by=['DISTANCE']) #sort the DISTANCE column
df_flights
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
bp = ax.boxplot(df_flights['DISTANCE'])

In [ ]:
#yousef khaled
# Remove outliers for DISTANCE
Q1 = df_flights['DISTANCE'].quantile(0.25)
Q3 = df_flights['DISTANCE'].quantile(0.75)
IQR = Q3 - Q1
filter_mask = (df_flights['DISTANCE'] >= Q1 - 1.5*IQR) & (df_flights['DISTANCE'] <= Q3 + 1.5*IQR)
outliersToBeDropped =df_flights.shape[0] - df_flights.loc[filter_mask].shape[0]
print(f'we will drop {outliersToBeDropped} rows')
df_flights = df_flights.loc[filter_mask]

There are some wrong values in the column "DEPARTURE_TIME", so lets fix them.


In [ ]:
#yousef khaled
df_flights["DEPARTURE_TIME"] = df_flights["WHEELS_OFF"] - df_flights["TAXI_OUT"]

Handle wrong calculations in "ELAPSED_TIME", "ARRIVAL_TIME", "ARRIVAL_DELAY", "DEPARTURE_DELAY", "AIR_TIME".


In [ ]:
#yousef khaled
df_flights["ELAPSED_TIME"] = df_flights["AIR_TIME"] + df_flights["TAXI_IN"] + df_flights["TAXI_OUT"]
df_flights["ARRIVAL_TIME"] = df_flights["WHEELS_ON"] + df_flights["TAXI_IN"]
df_flights["ARRIVAL_DELAY"] = df_flights["ARRIVAL_TIME"] - df_flights["SCHEDULED_ARRIVAL"]
df_flights["DEPARTURE_DELAY"] = df_flights["DEPARTURE_TIME"] - df_flights["SCHEDULED_DEPARTURE"]
df_flights["AIR_TIME"] = df_flights["WHEELS_ON"] - df_flights["WHEELS_OFF"]

In [ ]:
#yousef
# Boxplot before removing outliers for AIR_TIME
df_flights = df_flights.sort_values(by=['AIR_TIME']) #sort the AIR_TIME column
df_flights
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
bp = ax.boxplot(df_flights['AIR_TIME'])

In [ ]:
#yousef
# Remove outliers for AIR_TIME
Q1 = df_flights['AIR_TIME'].quantile(0.25)
Q3 = df_flights['AIR_TIME'].quantile(0.75)
IQR = Q3 - Q1
filter_mask = (df_flights['AIR_TIME'] >= Q1 - 1.5*IQR) & (df_flights['AIR_TIME'] <= Q3 + 1.5*IQR)
outliersToBeDropped =df_flights.shape[0] - df_flights.loc[filter_mask].shape[0]
print(f'we will drop {outliersToBeDropped} rows')
df_flights = df_flights.loc[filter_mask]

Integrating the flights with airlines where df_flights.AIRLINE -> df_airlines.IATA_CODE, one airline to many flights.

After that we don't need the IATA_CODE in our DF anymore as we have the airport name.

In [ ]:
df_integrated = pd.merge(df_flights, df_airlines, left_on="AIRLINE", right_on="IATA_CODE")
df_integrated.drop(['IATA_CODE','AIRLINE_x'], axis=1, inplace=True)
df_integrated.rename(columns={"AIRLINE_y": "AIRLINE"}, inplace =True)

In [ ]:
display(df_integrated)

Integrate result with airports.
df_flights.ORIGIN_AIRPORT, df_flights.DESTINATION_AIRPORT -> df_airports.IATA_CODE, one airport to many flights.

In [ ]:
# integrate with the source airport first.
df_integrated_origin = pd.merge(df_integrated, df_airports, left_on="ORIGIN_AIRPORT", right_on="IATA_CODE")
df_integrated_origin.drop(['ORIGIN_AIRPORT','IATA_CODE'], axis=1, inplace=True)

In [ ]:
display(df_integrated_origin)

Rename the inserted columns of the airport in integrated dataframe with origin_{column_name}

In [ ]:
df_integrated_origin.rename(columns={"AIRPORT": "ORIGIN_AIRPORT", "CITY": "ORIGIN_CITY", "STATE": "ORIGIN_STATE", "LATITUDE": "ORIGIN_LATITUDE", "LONGITUDE": "ORIGIN_LONGITUDE"}, inplace=True)

Integrate with the destination airport.

In [ ]:
df_full_integration = pd.merge(df_integrated_origin, df_airports, left_on="DESTINATION_AIRPORT", right_on="IATA_CODE")
df_full_integration.drop(['DESTINATION_AIRPORT','IATA_CODE'], axis=1, inplace=True)

In [ ]:
df_full_integration.rename(columns={"AIRPORT": "DEST_AIRPORT", "CITY": "DEST_CITY", "STATE": "DEST_STATE", "LATITUDE": "DEST_LATITUDE", "LONGITUDE": "DEST_LONGITUDE"}, inplace=True)
display(df_full_integration)

In [ ]:
df_full_integration.columns

Feature Engineering
--

We will add new Feature ( average speed of the plane = DISTANCE / AIR_TIME).

In [ ]:
dfi = df_full_integration

In [ ]:
dfi.loc[dfi['AIR_TIME']!=0, 'AVG_SPEED'] = dfi['DISTANCE'] / dfi['AIR_TIME']
dfi.loc[dfi['AIR_TIME']==0, 'AVG_SPEED'] = 0

In [ ]:
dfi.columns

Change cancellation_reason from categorical to numerical to be able to use it in analysis or put the data in ML model.
A:1, B:2, C:3, D:4, null: 0

In [ ]:
dfi.loc[dfi['CANCELLATION_REASON'] == 'A', 'CANCELLATION_REASON'] = 1
dfi.loc[dfi['CANCELLATION_REASON'] == 'B', 'CANCELLATION_REASON'] = 2
dfi.loc[dfi['CANCELLATION_REASON'] == 'C', 'CANCELLATION_REASON'] = 3
dfi.loc[dfi['CANCELLATION_REASON'] == 'D', 'CANCELLATION_REASON'] = 4
dfi['CANCELLATION_REASON'] = dfi['CANCELLATION_REASON'].fillna(0)

What is the average arrival_delay in each airline?

In [ ]:
sns.barplot(data=dfi.loc[dfi['ARRIVAL_DELAY'] > 0], x="ARRIVAL_DELAY", y="AIRLINE")

What are the airlines which have small delays, medium delays, high delays or come earlier than scheduled ?

Let's categorize the delays in each airline to 3:Early(<0) 0:low(0-10) 1:medium(10-30) 2:high(30+) delays

In [ ]:
dfi.loc[ (dfi['ARRIVAL_DELAY']>=0) & (dfi['ARRIVAL_DELAY']<=10), 'DELAY_CATEGORY'] = 0
dfi.loc[(dfi['ARRIVAL_DELAY']>10) & (dfi['ARRIVAL_DELAY']<=30), 'DELAY_CATEGORY'] = 1
dfi.loc[(dfi['ARRIVAL_DELAY']>30), 'DELAY_CATEGORY'] = 2
dfi.loc[(dfi['ARRIVAL_DELAY']<0), 'DELAY_CATEGORY'] = 3

Number of flights in each airline

In [ ]:
fig = plt.figure(1, figsize=(10,10))
ax=sns.countplot(y='AIRLINE', hue='DELAY_CATEGORY', data=dfi)

Let's see it without the early flights.

In [ ]:
ax=sns.countplot(y='AIRLINE', hue='DELAY_CATEGORY', data=dfi[(dfi['DELAY_CATEGORY']!=3)])

Insights about the arrival delays.
--

Southwest, Delta and American are the most 3 that come earlier than scheduled.

Southwest, Delta and American are the most 3 that have small delays ( delays<=10 mins )

Southwest, American and Delta are the most 3 that have long delays (delays > 30 mins)

Pearson correlation between numerical attributes
--

If some depends on the other/ Feature extraction / dimensionality reduction.

In [ ]:
# first select only the numerical columns
numerical = dfi.select_dtypes(include=np.number)

In [ ]:
# then visualize the correlation between the numerical attributes.

corr = dfi.corr()
fix, ax = plt.subplots(figsize=(12,12))
ax.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns, rotation='vertical')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.show()

When does the weather delay happens the most? or Does the month affects/have relation with the weather delays? 

In [ ]:
dfi.loc[dfi['WEATHER_DELAY']!=0, dfi.columns[22:28]]

In [ ]:
sns.barplot(data=dfi.loc[dfi['WEATHER_DELAY'] > 0], x="MONTH", y="WEATHER_DELAY")

December,November and may are the months which most happen the weather delays, but the month doesn't really affects the weather_delay of a flight.

Now does really the month affects the flights with cancellation? it may be cancelled due to weather in some months more than others.

group the data by the months.

In [ ]:
dfi[(dfi['CANCELLATION_REASON']==1)]

Filter the data that have the cancellation_reason is weather and plot against the month to see which months can affect the cancellation with the weather condition. 

In [ ]:
sns.countplot(data=dfi.loc[dfi['CANCELLATION_REASON'] == 2], x="MONTH")

December, January, February and March (Winter season) are the months where the flights have cancellation due to weather, try to travel in other months please :D.

Does some airlines have cancellation due to airline more than the others?

In [ ]:
sns.countplot(data=dfi.loc[dfi['CANCELLATION_REASON'] == 1], y="AIRLINE")

Yes: Southwest, Atlantic and Skywest are the most 3 airlines which cancel due to Airline/Carrier.

What is the relationship between MONTH and WEATHER_DELAY.

In [ ]:
#yousef
month_weather_plot = dfi.loc[:, ['MONTH', 'WEATHER_DELAY']].groupby('MONTH').mean()
month_weather_plot

In [ ]:
#yousef
month_weather_plot.plot.bar(figsize=(12,8), stacked=False)
plt.show()



We can see from the bar chart that Febuary has the most weather delays because of the bad weather.


Create new feature called "TOTAL_AIRLINE_DELAY" that is the summation of all delays related to an airline (AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY)


In [ ]:
#yousef
dfi['TOTAL_AIRLINE_DELAY'] = dfi['AIR_SYSTEM_DELAY'] + dfi['SECURITY_DELAY'] 
+ dfi['AIRLINE_DELAY'] + dfi['LATE_AIRCRAFT_DELAY'] + dfi['WEATHER_DELAY']

Let's see each Airline's performance by looking at the newly created feature "TOTAL_AIRLINE_DELAY".


In [ ]:
#yousef
airline_delay_plot = dfi.loc[:, ['AIRLINE','TOTAL_AIRLINE_DELAY']].groupby('AIRLINE').mean()
airline_delay_plot

In [ ]:
#yousef
airline_delay_plot.plot.bar(figsize=(12,8), stacked=False)
plt.show()



The new feature TOTAL_AIRLINE_DELAY helped us evaluate the performance of each airline.

Hawaiian Airlines Inc is the best performing airline




Which airlines have the most canceled flights?


In [ ]:
#yousef
sns.countplot(data=dfi.loc[dfi['CANCELLATION_REASON'] != 0], y="AIRLINE")